In [1]:
import mysql.connector

1. **Task: Bank Transactions**
You are tasked with implementing a banking system that allows customers to transfer funds between accounts.
Design a SQL transaction to ensure that the amount is deducted from the sender's account and added to the receiver's 
account in an atomic manner.
If any part of the transaction fails, rollback the changes to maintain data integrity.

In [6]:
# Lets imagine this kind of table in DB:

# CREATE TABLE customers_accounts (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     account_number VARCHAR(20),
#     balance DECIMAL(10, 2)
# );

# INSERT INTO customers_accounts (account_number, balance) VALUES
# ('A123456', 1000),
# ('B987654', 750),
# ('C456789', 1500),
# ('D111222', 300),
# ('E555666', 2000);

In [4]:
connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
cursor = connection.cursor()

# Get sender's current balance
cursor.execute("SELECT balance FROM customers_accounts WHERE account_number = 'A123456'")
sender_balance = cursor.fetchone()[0]

# Check if the sender has enought money in his balance
if sender_balance < 1000:
    print("There is not enough money in senders balance")
    connection.rollback()
else:
    # reduce sender's balance
    cursor.execute("UPDATE customers_accounts SET balance = balance - 100  WHERE account_number = 'A123456'")

    # increase receiver's balance
    cursor.execute("UPDATE customers_accounts SET balance = balance + 100 WHERE account_number = 'B987654'")
    connection.commit()
    print("Transaction completed!")

cursor.close()
connection.close()

There is not enough money in senders balance


True

2. **Task: Order Processing**
You are developing an online shopping platform.
Create a SQL transaction that updates the inventory of products when a customer places an order.
Deduct the ordered quantity from the product stock and update the order status.
If any part of the transaction fails (e.g., insufficient stock), roll back the changes to ensure consistent data.

In [ ]:
# CREATE TABLE products (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     product_name VARCHAR(100),
#     stock_quantity INT
# );

# CREATE TABLE orders (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     customer_name VARCHAR(100),
#     product_id INT,
#     order_quantity INT,
#     order_status VARCHAR(20)
# );

# INSERT INTO products (product_name, stock_quantity) VALUES
# ('Product 1', 100),
# ('Product 2', 50),
# ('Product 3', 200),
# ('Product 4', 75),
# ('Product 5', 300);

# INSERT INTO orders (customer_name, product_id, order_quantity, order_status) VALUES
# ('John Doe', 1, 10, 'Processing'),
# ('Alice Smith', 3, 5, 'Processing'),
# ('Bob Johnson', 2, 20, 'Processing'),
# ('Eva Brown', 4, 15, 'Processing'),
# ('Michael Davis', 5, 25, 'Processing');

In [5]:
connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
cursor = connection.cursor()

# get the amount of the product in stock 
cursor.execute("SELECT stock_quantity FROM products WHERE id = 2")
product_stock = cursor.fetchone()

# check if it is enough
if 45 > product_stock[0]:
    print("There are not enough products in stock")
    connection.rollback()
else: 
    # Deduct the ordered quantity 
    cursor.execute("UPDATE products SET stock_quantity = stock_quantity- 45 WHERE id = 2")

    # Update the order status
    cursor.execute("INSERT INTO orders (customer_name, product_id, order_quantity, order_status) "
                    "VALUES ('Bob Johnson', 2, 10, 'Completed')")

    connection.commit()
    print("Order placed successfully!")

cursor.close()
connection.close()

There are not enough products in stock


True

3. **Task: Employee Salary Adjustment**
Your HR department needs to apply a salary adjustment to all employees based on their performance.
Design a SQL transaction that updates the salary of each employee with the corresponding adjustment and logs the changes. 
If there is any issue while updating salaries, roll back the changes to maintain the previous salary data.

In [ ]:
# CREATE TABLE employees (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     employee_name VARCHAR(100),
#     salary DECIMAL(10, 2)
# );

# CREATE TABLE salary_adjustments (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     employee_id INT,
#     adjustment DECIMAL(10, 2),
#     adjustment_reason VARCHAR(255),
#     adjusted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# );

# INSERT INTO employees (employee_name, salary) VALUES
# ('John Doe', 50000.00),
# ('Alice Smith', 60000.00);

In [6]:
connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
try: 
    cursor = connection.cursor()
    
    employee_id = 2
    adjustment = 400
    
    # get current salary
    cursor.execute("SELECT salary FROM employees WHERE id = %s", (employee_id,))
    current_salary = cursor.fetchone()[0]

    # Calculate the new salary 
    new_salary = current_salary + adjustment

    # Update the salary in the employee table
    cursor.execute("UPDATE employees SET salary = %s WHERE id = %s", (new_salary, employee_id))

    # Log the changes of salary 
    cursor.execute("INSERT INTO salary_adjustments (employee_id, adjustment, adjustment_reason) "
                           "VALUES (%s, %s, 'Salary Adjustment')", (employee_id, adjustment))


    connection.commit()
    print("Salary updated")
    
# If there is any issue while updating salaries, rollback the changes to maintain the previous salary data
except (Exception, mysql.connector.Error) as error:
    connection.rollback()
    print("Salary updates failed")
    
finally:
    cursor.close()
    connection.close()

Salary updated


4. **Task: Flight Reservation**
You are building a flight reservation system. Create a SQL transaction that reserves a seat for a passenger on a flight.
Deduct the seat availability, update the reservation record, and log the booking details.
If any part of the transaction fails (e.g., seat already taken), roll back the changes and notify the user of the failure.

In [ ]:
# CREATE TABLE flights (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     flight_number VARCHAR(20),
#     total_seats INT,
#     available_seats INT
# );

# CREATE TABLE bookings (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     flight_id INT,
#     passenger_id INT,
#     seat_number INT,
#     booking_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# );

In [8]:
connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
cursor = connection.cursor()

flight_id = 2
passenger_id = 2
seat_number = 8

# Check available seats
cursor.execute("SELECT available_seats FROM flights WHERE id = %s", (flight_id,))
available_seats = cursor.fetchone()[0]

# Check if the seat is available
cursor.execute("SELECT COUNT(*) FROM bookings WHERE flight_id = %s AND seat_number = %s", (flight_id, seat_number))
seat_taken = cursor.fetchone()[0]

if available_seats <= 0 or seat_taken > 0:
    print("This seat is not available")
    connection.rollback()
else: 
    # Deduct the seat availability
    new_available_seats = available_seats - 1
    cursor.execute("UPDATE flights SET available_seats = %s WHERE id = %s", (new_available_seats, flight_id))

    # Reserve the seat for the passenger
    cursor.execute("INSERT INTO bookings (flight_id, passenger_id, seat_number) "
                       "VALUES (%s, %s, %s)", (flight_id, passenger_id, seat_number))

    connection.commit()
    print("seat reserved")

cursor.close()
connection.close()

This seat is not available


True

5. **Task: Product Returns**
You are developing an e-commerce platform. Implement a SQL transaction to process product returns initiated by customers.
Update the product's return status, refund the customer's payment, and log the return details.
If any part of the transaction fails (e.g., invalid return request), roll back the changes and handle the error gracefully.

In [ ]:
# CREATE TABLE return_logs (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     customer_id INT NOT NULL,
#     product_id INT NOT NULL,
#     refund_amount DECIMAL(10, 2) NOT NULL,
#     return_date timestamp NOT NULL,
#     status VARCHAR(50)
# );

In [17]:
try:
    connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
    cursor = connection.cursor()
    
    insert_data = {
    'customer_id': 34,
    'product_id': 14,
    'refund_amount': 100.00
    }

    # Insert the return details
    cursor.execute("INSERT INTO return_logs (customer_id, product_id, refund_amount, return_date, status)"
                    "VALUES (%s, %s, %s, NOW(), %s)", 
                   (insert_data['customer_id'], insert_data['product_id'], insert_data['refund_amount'], 'completed'))

    connection.commit()
    print("Transaction completed.")

except mysql.connector.Error as error:
    connection.rollback()
    print("Transaction failed. Error:", error)

finally:
    cursor.close()
    connection.close()

Transaction completed.


6. **Task: Student Enrollment**
Design a SQL transaction to handle the enrollment of new students in a university.
Update the student enrollment records, assign courses, and update the tuition fee payment status. 
If any part of the transaction fails (e.g., course limit reached), roll back the changes to ensure no invalid enrollments are recorded.

In [ ]:
# CREATE TABLE enrollment_records (
#     enrollment_id INT AUTO_INCREMENT PRIMARY KEY,
#     student_id INT NOT NULL,
#     course_id INT NOT NULL,
#     payment_status VARCHAR(50) DEFAULT 'pending'
# );

In [21]:
try:
    connection = mysql.connector.connect(user="root",
                                     password="psw",
                                     host="localhost",
                                     database="db")
    cursor = connection.cursor()
    
    # Example data for student enrollment
    student_id = 122
    course_ids = [101, 102, 103]  # List of course IDs 

    
    # Check if the student is already enrolled in any of the selected courses
    for course_id in course_ids:
        cursor.execute("SELECT enrollment_id FROM enrollment_records WHERE student_id = %s AND course_id = %s", (student_id, course_id))
        existing_enrollment = cursor.fetchone()
        if existing_enrollment:
            raise Exception("Student has already enrolled in the course")

    # Insert new enrollment records
    for course_id in course_ids:
        cursor.execute("INSERT INTO enrollment_records (student_id, course_id) VALUES (%s, %s)", (student_id, course_id))

    # Update payment status 
    for course_id in course_ids:
        cursor.execute("UPDATE enrollment_records SET payment_status = 'paid' WHERE student_id = %s AND course_id = %s", (student_id, course_id))

    connection.commit()
    print("Enrollment process completed successfully.")

except mysql.connector.Error as error:
    connection.rollback()
    print("Enrollment process failed. Error:", error)

finally:
    cursor.close()
    connection.close()

Enrollment process completed successfully.


Non SQL database:

In Python, design your own design of the database (using the files) and perform some practical examples with it. 

In [78]:
import json


In [79]:
book = {"id":1,"title":"book1","author":"authorA","year":1924}

books = list()
books.append(book)

In [80]:
with open("books.json","w") as json_f:
    json.dump(books,json_f)

In [81]:
class BooksDB:
    def __init__(self):
        self.data = self.load_data()
        
    def load_data(self):
        with open('books.json', 'r') as file:
            data = json.load(file)
        return data

    def save_data(self, data):
        with open('books.json', 'w') as file:
            json.dump(self.data, file, indent=4)

    def add_book(self, title, author, year):
        # Generate a new book ID
        new_id = 1
        if self.data:
            new_id = max(book['id'] for book in self.data) + 1

        # Add the new book to the database
        new_book = {
            'id': new_id,
            'title': title,
            'author': author,
            'year': year
        }
        self.data.append(new_book)

        # Save the updated data to the file
        self.save_data(self.data)

    # Find all books with the given author
    def get_books_by_author(self, author):
        author_books = [book for book in self.data if book['author'] == author]
        return author_books


In [82]:
book_db = BooksDB()

In [83]:
# Add new books to the database
book_db.add_book("Book2", "authorB", 2014)
book_db.add_book("Book3", "authorD", 2015)

In [84]:
# find books by author
author_a_books = book_db.get_books_by_author("authorA")
print("Books by authorA:")
for book in author_a_books:
    print(book)

Books by authorA:
{'id': 1, 'title': 'book1', 'author': 'authorA', 'year': 1924}
